In [1]:
import os
import glob
import pandas as pd
import numpy as np
from copy import deepcopy

In [4]:
# Paths
run_directory = r"C:\Users\pechen\Desktop\VTP\Model_Analysis"
run_folder = r"20191211_KK"
run_path = os.path.join(run_directory,run_folder)
# Zone Mapping Path
input_pth = r'C:\Users\pechen\Desktop\VTP\Model_Analysis'
zone_map = pd.read_csv(os.path.join(input_pth,"VTP_zone_mapping.csv"))
zone_map_min = zone_map

In [3]:
microsim_location = r"Microsim Results"
microsim_path = os.path.join(run_path, microsim_location)

persons_df = pd.read_csv(os.path.join(microsim_path,"persons.csv"))
trips_df = pd.read_csv(os.path.join(microsim_path,"trips.csv"))
tripmode_df = pd.read_csv(os.path.join(microsim_path,"trip_modes.csv"))
trips_gta = trips_df[(trips_df.o_zone < 6000) & (trips_df.d_zone < 6000)]

trips_gta = trips_gta.merge(persons_df,on=["household_id","person_id"])
trips_gta = trips_gta[trips_gta.age >=11]

trips_gta_time = tripmode_df.merge(trips_gta,how = "right", on = ['household_id', 'person_id', 'trip_id'])

trips_gta_time["weight"] = trips_gta_time["weight"]/10
trips_gta_time["weight_z"] = trips_gta_time["weight"]*trips_gta_time["weight_x"]

ms_daily_GTA = pd.pivot_table(trips_gta_time,columns = "mode", values = "weight_z", aggfunc = sum)


In [5]:
ms_daily_GTA

mode,Auto,Bicycle,Carpool,DAT,PAT,PET,Passenger,RideShare,Schoolbus,VFH,WAT,Walk
weight_z,6815007.0,146210.7,479109.0,355325.0,248813.3,208378.3,596124.3,183995.8,245489.1,554694.8,1927641.3,1443167.2


In [ ]:
ms_daily_GTA.to_csv(os.path.join(input_pth,"GTA_Wide_TripsbyMode_MCSpatial_Run41.csv")) #Daily MSa

In [ ]:
trips_407 = trips_df
trips_407  = trips_407.merge(persons_df,on=["household_id","person_id"])
trips_407 = trips_407[trips_407.age >=11]
trips_407 = trips_407.merge(zone_map_min,left_on = "o_zone", right_on ="TAZ").drop(columns = "TAZ").rename(columns = {"PD":"PD_o"})
trips_407 = trips_407.merge(zone_map_min,left_on = "d_zone", right_on ="TAZ").drop(columns = "TAZ").rename(columns = {"PD":"PD_d"})

trips_407_time = tripmode_df.merge(trips_407,how = "right", on = ['household_id', 'person_id', 'trip_id'])
trips_407_time["weight"] = trips_407_time["weight"]/10
trips_407_time["weight_z"] = trips_407_time["weight"]*trips_407_time["weight_x"]
trips_407_am = trips_407_time[(trips_407_time.o_depart >= 360) & (trips_407_time.o_depart < 540)]
trips_407_pm = trips_407_time[(trips_407_time.o_depart >= 900) & (trips_407_time.o_depart < 1140)]



In [7]:
am_modeshare = pd.pivot_table(trips_407_am,index = ["PD_o","PD_d"],columns = "mode", values = "weight_z", aggfunc = sum,margins = True).fillna(0)
pm_modeshare = pd.pivot_table(trips_407_pm,index = ["PD_o","PD_d"],columns = "mode", values = "weight_z", aggfunc = sum,margins = True).fillna(0)

In [36]:
am_modeshare = 

,household_id,person_id,trip_id,mode,o_depart,d_arrive,weight,o_act,o_zone,d_act,...,work_zone,school_zone,weight_y,GTAM_TAZ_x,PD_o,REGION_x,GTAM_TAZ_y,PD_d,REGION_y,weight_z
0,5,1,1,WAT,375.2311,435.0000,0.6,Home,1,PrimaryWork,...,57,0,1,1,1,1,57,1,1,0.6
1,5,1,1,Bicycle,409.2779,435.0000,0.3,Home,1,PrimaryWork,...,57,0,1,1,1,1,57,1,1,0.3
2,5,1,1,VFH,411.0402,435.0000,0.1,Home,1,PrimaryWork,...,57,0,1,1,1,1,57,1,1,0.1
4,67,1,1,WAT,532.3185,570.0000,0.2,Home,1,IndividualOther,...,0,0,1,1,1,1,57,1,1,0.2
8,103,1,1,WAT,398.2448,458.0136,0.3,Home,1,PrimaryWork,...,57,0,1,1,1,1,57,1,1,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30448461,2340313,2,1,Carpool,471.6019,480.0000,0.1,Home,5120,PrimaryWork,...,5021,0,1,5120,46,6,5021,44,6,0.1
30448462,2462578,2,1,WAT,488.2504,570.0000,0.1,Home,5243,Market,...,0,0,1,5243,44,6,5021,44,6,0.1
30448465,2462578,2,1,Walk,512.4098,570.0000,0.1,Home,5243,Market,...,0,0,1,5243,44,6,5021,44,6,0.1
30448477,1154556,2,1,Auto,410.0223,420.0000,0.9,Home,1100,Market,...,1100,0,1,100,22,2,158,23,2,0.9


In [14]:
am_modeshare.to_csv(os.path.join(input_pth,"am_modeshare_MCSpatial_Run41.csv")) #Time Period MS
pm_modeshare.to_csv(os.path.join(input_pth,"pm_modeshare_MCSpatial_Run41.csv"))